In [11]:
# use Summary of weather.csv
"https://www.kaggle.com/code/rtatman/datasets-for-regression-analysis/notebook"

'https://www.kaggle.com/code/rtatman/datasets-for-regression-analysis/notebook'

In [12]:
import pandas as pd
import tensorflow as tf
import seaborn as sns

In [13]:
dataset = pd.read_csv('./datasets/weatherHistory 2.csv')
dataset.head()

,Formatted Date,Summary,Precip Type,Temperature (C),Apparent Temperature (C),Humidity,Wind Speed (km/h),Wind Bearing (degrees),Visibility (km),Loud Cover,Pressure (millibars),Daily Summary
0,2006-04-01 00:00:00.000 +0200,Partly Cloudy,rain,9.472222,7.388889,0.89,14.1197,251.0,15.8263,0.0,1015.13,Partly cloudy throughout the day.
1,2006-04-01 01:00:00.000 +0200,Partly Cloudy,rain,9.355556,7.227778,0.86,14.2646,259.0,15.8263,0.0,1015.63,Partly cloudy throughout the day.
2,2006-04-01 02:00:00.000 +0200,Mostly Cloudy,rain,9.377778,9.377778,0.89,3.9284,204.0,14.9569,0.0,1015.94,Partly cloudy throughout the day.
3,2006-04-01 03:00:00.000 +0200,Partly Cloudy,rain,8.288889,5.944444,0.83,14.1036,269.0,15.8263,0.0,1016.41,Partly cloudy throughout the day.
4,2006-04-01 04:00:00.000 +0200,Mostly Cloudy,rain,8.755556,6.977778,0.83,11.0446,259.0,15.8263,0.0,1016.51,Partly cloudy throughout the day.


In [14]:
X = dataset["Temperature (C)"].values
y = dataset["Humidity"].values
X

array([ 9.47222222,  9.35555556,  9.37777778, ..., 22.03888889,
       21.52222222, 20.43888889])

In [15]:
X.ndim, y.ndim

(1, 1)

In [16]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=42,test_size=0.2)

In [17]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(20),
    tf.keras.layers.Dense(1)
])
model.compile(
    loss="mse",
    optimizer=tf.keras.optimizers.Adam(lr=0.000001),
    metrics=["mse","mae"]
)
callback = tf.keras.callbacks.EarlyStopping(monitor="loss",patience=3)
model.fit(tf.expand_dims(X_train,axis=-1),y_train, epochs=100,callbacks=[callback])

Epoch 1/100


/Users/kingchan818/miniforge3/envs/tf/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)
2022-06-30 23:20:15.503868: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


2412/2412 [==============================] - 11s 5ms/step - loss: 684.3246 - mse: 684.3246 - mae: 21.5446
Epoch 2/100
2412/2412 [==============================] - 11s 5ms/step - loss: 666.0721 - mse: 666.0721 - mae: 21.2523
Epoch 3/100
2412/2412 [==============================] - 12s 5ms/step - loss: 648.2773 - mse: 648.2773 - mae: 20.9632
Epoch 4/100
2412/2412 [==============================] - 11s 4ms/step - loss: 630.8972 - mse: 630.8972 - mae: 20.6771
Epoch 5/100
2412/2412 [==============================] - 10s 4ms/step - loss: 613.9164 - mse: 613.9164 - mae: 20.3938
Epoch 6/100
2412/2412 [==============================] - 11s 4ms/step - loss: 597.3094 - mse: 597.3094 - mae: 20.1129
Epoch 7/100
2412/2412 [==============================] - 11s 4ms/step - loss: 581.0490 - mse: 581.0490 - mae: 19.8340
Epoch 8/100
2412/2412 [==============================] - 11s 5ms/step - loss: 565.1948 - mse: 565.1948 - mae: 19.5580
Epoch 9/100
2412/2412 [==============================] - 11s 5ms/ste